In [1]:
import os
import seaborn as sns
import copy

In [2]:
# Add current working directory to Python's sys path if not included
imp_mods = os.sys.path
my_pkg = os.getcwd()

if my_pkg not in imp_mods:
    imp_mods.append(my_pkg)
    print('Package imported.')

In [3]:
# import class CensusDataset from a customized module census_methods
from census_methods import CensusDataset as cd

In [4]:
# read-in data from csv file
census_df = sns.categorical.pd.read_csv('census_16_new - Copy.csv')

In [5]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7581 entries, 0 to 7580
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   House Number                   7579 non-null   object
 1   Street                         7581 non-null   object
 2   First Name                     7579 non-null   object
 3   Surname                        7581 non-null   object
 4   Age                            7581 non-null   object
 5   Relationship to Head of House  7581 non-null   object
 6   Marital Status                 5904 non-null   object
 7   Gender                         7581 non-null   object
 8   Occupation                     7581 non-null   object
 9   Infirmity                      7581 non-null   object
 10  Religion                       5864 non-null   object
dtypes: object(11)
memory usage: 651.6+ KB


___

### House Number

In [6]:
# create a copy of house number series to avoid making
# unwanted changes to the main dataframe
house_num = copy.deepcopy(census_df['House Number'])

In [7]:
# check for empty string
cd.check_for_empty_str(census_df)['House Number']

False

**NOTE:**<br>
No blanks in the House Number column<br><br>

In [8]:
# check for nan values
cd.null_checker(house_num)

2

**NOTE:**<br>
There are two nan values in the House Number column<br><br>

In [9]:
# unique values per column
print(house_num.unique())

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111' '112'
 '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123' '124'
 '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135' '136'
 '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147' '148'
 '149' '150' 'Two' 'One' nan]


<br><br>Before we can change the datatype of house number to int, we must first make sure that all of its values are convertible to integers

#### Change spellings to numbers

In [10]:
converter = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
house_num = cd.transform_val(house_num, converter)

In [11]:
# look at owners of missing house numbers
null_house_num = census_df.loc[house_num.isnull()]
null_house_num

,House Number,Street,First Name,Surname,Age,Relationship to Head of House,Marital Status,Gender,Occupation,Infirmity,Religion
7366,NaN,Chester Lane,Dominic,Murphy,10,Son,NaN,Male,Student,None,NaN
7445,NaN,Chester Lane,Charlene,Bibi,61,Head,Married,Female,Planning and development surveyor,None,None


<br><br>The two people with missing house numbers seem to live on the same street. While one is the head, the other is a son.<br>
Let's check for other occupants of the same house holds using Street and Surname

In [12]:
surnames = null_house_num['Surname'].values
streets = null_house_num['Street'].values
census_df.loc[(census_df['Surname'].isin(surnames)) & (census_df['Street'].isin(streets))]

,House Number,Street,First Name,Surname,Age,Relationship to Head of House,Marital Status,Gender,Occupation,Infirmity,Religion
7364,13,Chester Lane,Gregory,Murphy,43,Head,Married,Male,"Secretary, company",None,None
7365,13,Chester Lane,Abbie,Murphy,39,Wife,Married,Female,Landscape architect,None,None
7366,NaN,Chester Lane,Dominic,Murphy,10,Son,NaN,Male,Student,None,NaN
7367,13,Chester Lane,Catherine,Murphy,8,Daughter,NaN,Female,Student,None,NaN
7444,35,Chester Lane,Francis,Murphy,56,Head,Single,Male,"Journalist, newspaper",None,Methodist
7445,NaN,Chester Lane,Charlene,Bibi,61,Head,Married,Female,Planning and development surveyor,None,None
7446,36,Chester Lane,Abdul,Bibi,60,Husband,Married,Male,Engineering geologist,None,None
7447,36,Chester Lane,Andrew,Bibi,33,Son,Single,Male,"Research officer, trade union",None,None
7448,36,Chester Lane,Kelly,Bibi,29,Daughter,Single,Female,Environmental health practitioner,None,None


<br><br>We can see that the Murphys are a household of 4 occupants (at house number: 13): Husband/Father, Wife/Mother, Son and Daughter.<br>
Albeit, there is a fifth person with surname Murphy, however, he has a different house number from the other Murphys and single.<br>Likewise for the Bibis, they are also a household of 4 occupants (at house number: 36): Head/Wife, Husband, Son and Daughter.<br>***Thus, a reasonable assumption is that Dominic Murphy's house number is 13. <br>And Charlene Bibi's house number is 36***

In [13]:
# assign Dominic Murphy, a house number 13
house_num.loc[null_house_num.loc[null_house_num['Surname'] == 'Murphy'].index] = str(13)

In [14]:
# assign Charlene Bibi, a house number 36
house_num.loc[null_house_num.loc[null_house_num['Surname'] == 'Bibi'].index] = str(36)

In [15]:
# check for replacement
house_num.loc[null_house_num.index]

7366    13
7445    36
Name: House Number, dtype: object

In [16]:
# check for missing values
print(house_num.unique())

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111' '112'
 '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123' '124'
 '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135' '136'
 '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147' '148'
 '149' '150']


#### The House number series now looks ready to be casted to type int

In [17]:
# cast house_num series to type int
house_num = sns.categorical.pd.to_numeric(house_num)

In [18]:
print(house_num.unique())

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150]


#### Replace the old "House Number" column in the original census_df dataframe with cleaned series, "house_num".

In [19]:
census_df['House Number'] = house_num

In [20]:
census_df.dtypes

House Number                      int64
Street                           object
First Name                       object
Surname                          object
Age                              object
Relationship to Head of House    object
Marital Status                   object
Gender                           object
Occupation                       object
Infirmity                        object
Religion                         object
dtype: object

___

### Street

In [21]:
# check for nan values
cd.null_checker(census_df['Street'])

0

In [22]:
# check for empty string
cd.check_for_empty_str(census_df)['Street']

False

In [23]:
# check out the unique values
print(cd.unique_categs(census_df)['Street'])

['Simmons Meadow', 'Morgan Avenue', 'Pillar Road', 'Wharf Place', 'Blue Spring', 'Stirling View', 'Smith Street', 'Griffiths Inlet', 'Shrine Harbor', 'Rees Locks', 'Dixon Drive', 'Howell Drive', 'Wharf Wells', 'Queen Avenue', 'Tegid Street', 'Chandler Drive', 'Adams Ways', 'Guardian Road', 'Runnymede Harbors', 'Coconutbelt Lane', 'North Road', 'Hall Lane', 'George Pass', 'Elizabeth Lane', 'Belle Center', 'Bell Orchard', 'Brooks Road', 'Windsor Avenue', 'Paladin Summit', 'Wright Brook', 'Green Wall', 'Doherty Extensions', 'Shillingcut Lane', 'February Street', 'Regent Lane', 'Mason Prairie', 'Scotland Avenue', 'Butterfly Avenue', 'Mary Drive', 'Mercia Underpass', 'Halfcrown Terrace', 'Oyster Road', 'Campbell Station', 'Parry Manor', 'Price Lane', 'Taylor Gateway', 'Brightwater Drive', 'Peacock Neck', 'Clayton Brooks', 'Portsmouth Park', 'Smith Manors', 'Spencer Avenue', 'Clementine Points', 'Guild Lane', 'Cancerpath Drive', 'Albion Springs', 'Duchess Street', 'Dockers Fords', 'Edwards F

**NOTE:**<br>
Street column does not require any cleaning, as there isn't any empty string, nan, nor inconsistent values in the column.

___

### Address

Create new "Address" feature by combining "House Number" and "Street" of each occupant<br>
**NOTE:** An "Address" feature would facilitate easier and faster access to unique households

In [24]:
address = census_df.apply(lambda df: f"{df['House Number']}, {df['Street']}", axis=1)
address

0       1, Simmons Meadow
1       1, Simmons Meadow
2       1, Simmons Meadow
3       1, Simmons Meadow
4       1, Simmons Meadow
              ...        
7576       1, Newry Spire
7577       1, Newry Spire
7578       1, Newry Spire
7579       1, Newry Spire
7580       1, Newry Spire
Length: 7581, dtype: object

#### Make address series a new column in the census_df

In [25]:
census_df['Address'] = address

___

#### checking the new structure of census_df

In [26]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7581 entries, 0 to 7580
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   House Number                   7581 non-null   int64 
 1   Street                         7581 non-null   object
 2   First Name                     7579 non-null   object
 3   Surname                        7581 non-null   object
 4   Age                            7581 non-null   object
 5   Relationship to Head of House  7581 non-null   object
 6   Marital Status                 5904 non-null   object
 7   Gender                         7581 non-null   object
 8   Occupation                     7581 non-null   object
 9   Infirmity                      7581 non-null   object
 10  Religion                       5864 non-null   object
 11  Address                        7581 non-null   object
dtypes: int64(1), object(11)
memory usage: 710.8+ KB


In [27]:
census_df.head()

,House Number,Street,First Name,Surname,Age,Relationship to Head of House,Marital Status,Gender,Occupation,Infirmity,Religion,Address
0,1,Simmons Meadow,Andrea,Bradley,43,Head,Married,Female,Prison officer,None,None,"1, Simmons Meadow"
1,1,Simmons Meadow,Peter,Bradley,44,Husband,Married,Male,Futures trader,None,None,"1, Simmons Meadow"
2,1,Simmons Meadow,Lauren,Bradley,5,Daughter,NaN,F,Student,None,NaN,"1, Simmons Meadow"
3,1,Simmons Meadow,Henry,Bradley,5,Son,NaN,Male,Student,None,NaN,"1, Simmons Meadow"
4,1,Simmons Meadow,Edward,Slater,25,Lodger,Divorced,Male,Further education lecturer,None,None,"1, Simmons Meadow"


___

#### Save dataframe changes to filesystem as csv file

In [28]:
fname = 'house_num_and_street_cleaned.csv'
census_df.to_csv(fname, index=False)

___

**<center> THE END</center>**

___